In [37]:
import pandas as pd
import os

sentences = []
for file in os.listdir('csv'):
    if file[-4:] == '.csv':
        errors = pd.read_csv(f'csv/{file}', encoding='ANSI', sep=';')
        correct_sentences = [correction.split('\n')[0].replace('’', '\'') for correction in errors['correct sentence']]
        sentences += correct_sentences

print(sentences[:10])
print(type(sentences))

['We have nice weather.', 'I consider myself someone who enjoys nature.', "She's a nice person in real life.", 'When you talk to other people, you have to be able to understand different accents.', 'I thought it would be a piece of cake.', 'It was normal when we were children.', 'My mother always watches football on TV.', 'My family has a lot of problems with health.', 'For Chinese people, socialism is like the monarchy in the U.K.', "He's allergic to dogs."]
<class 'list'>


In [38]:
import spacy

nlp = spacy.load('en_core_web_sm')

parsed_sentences = [nlp(sentence) for sentence in sentences]


In [39]:
import random
import pyinflect

def get_ger_inf_list(num_sentences):
    ger_inf_list = []
    seen_sentences = []
    while len(ger_inf_list) < num_sentences:
        random_sentence = sentences[random.choice(range(len(sentences)-1))]
        if random_sentence not in seen_sentences:
            seen_sentences.append(random_sentence)
            parsed_sentence = nlp(random_sentence)
            word_list = []
            
            for token in parsed_sentence:
                if token.tag_ == 'TO':
                    next = parsed_sentence[token.i+1] if token.i+1 < len(parsed_sentence) else None
                    if next and next.tag_ == 'VB':
                        continue
                elif token.tag_ == 'VB':
                    prev = parsed_sentence[token.i-1] if token.i-1 >= 0 else None
                    if prev and prev.tag_ == 'TO':
                        inflected_forms = ['to ' + token.text] + get_verb_forms(token, ['VBG', 'TO'])
                        word_list.append(inflected_forms)
                    else:
                        word_list.append([token.text])
                elif token.tag_ == 'VBG':
                    prev = parsed_sentence[token.i-1] if token.i-1 >= 0 else None
                    if (prev and prev.lemma_ == 'be') or (prev and prev.text == 'n\'t'):
                        word_list.append([token.text])
                    else:
                        inflected_forms = [token.text] + get_verb_forms(token, ['VBG', 'TO'])
                        word_list.append(inflected_forms)
                else:
                    word_list.append([token.text])
            for word in word_list:
                if len(word) > 1:
                    ger_inf_list.append(word_list)
                    break

    return ger_inf_list

def get_verb_forms(token, target_tags):
    inflected_forms = []
    for tag in target_tags:
        if tag == 'TO':
            infinitive = token._.inflect('VB')
            if infinitive:
                inflected_forms.append('to ' + infinitive)
            else:
                inflected_forms.append('to ' + token.text)
        else:
            inflected_form = token._.inflect(tag)
            if inflected_form:
                inflected_forms.append(inflected_form)
    return inflected_forms

print(get_ger_inf_list(10))

[[['I (I)'], ['thought (think)'], ['about (about)'], ['moving', 'moving', 'to move'], ['to (to)'], ['Australia (Australia)'], ['. (.)']], [['I (I)'], ['prefer (prefer)'], ['playing', 'playing', 'to play'], ['/ (/)'], ['to play', 'playing', 'to play'], ['team (team)'], ['sports (sport)'], ['. (.)']], [['It (it)'], ["'s (be)"], ['easier (easy)'], ['to understand', 'understanding', 'to understand'], ['. (.)']], [['It (it)'], ["'s (be)"], ['very (very)'], ['expensive (expensive)'], ['to live', 'living', 'to live'], ['in (in)'], ['Norway (Norway)'], ['. (.)']], [['I (I)'], ['always (always)'], ['think (think)'], ['about (about)'], ['donating', 'donating', 'to donate'], ['blood (blood)'], ['. (.)']], [['I (I)'], ['always (always)'], ['spend (spend)'], ['a (a)'], ['lot (lot)'], ['of (of)'], ['money (money)'], ['trying', 'trying', 'to try'], ['to improve', 'improving', 'to improve'], ['my (my)'], ['skin (skin)'], ['. (.)']], [['I (I)'], ["'m (be)"], ['here (here)'], ['to be', 'being', 'to be']